# Advanced Image Classification with Lightweight CNN

This notebook demonstrates advanced image classification using a lightweight convolutional neural network (CNN) model trained on the CIFAR-10 dataset. The model includes additional layers and techniques for improved performance.

## Instructions

1. Run each cell sequentially to execute the code.
2. Make sure to read the comments provided in the code cells for additional information and instructions.
3. Experiment with different hyperparameters, model architectures, and techniques to improve performance. Consider:
   - **Experiment with Model Architecture**: Try different architectures for the Lightweight CNN model. You can increase the depth, width, or add additional layers like Batch Normalization, Dropout or Residual Connections to improve performance.
4. Consider Using a deeper CNN architecture, Adding Batch Normalization layers to stabilize and accelerate training, applying Dropout regularization to prevent overfitting, and using data augmentation techniques for training data. You may also test to train the model for more / less epochs and find a proper moment for learning rate decay.
5. Create a ResNet-50 model with ImageNet pre-trained parameters. Evaluate the classification capability of the pre-trained model on a 10-category test dataset.

In [ ]:
# Install necessary packages
!pip install torch torchvision matplotlib

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Step 1: Data Preprocessing and Augmentation

Load the CIFAR-10 dataset, apply transformations for data preprocessing and augmentation, and create data loaders for training and validation.

In [ ]:
# Define data transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

## Step 2: Define the Lightweight CNN Model

Define a lightweight CNN model for image classification with additional layers and techniques for improved performance.

In [ ]:
import torch.nn.functional as F
# Define the CNN model
class LightweightCNN(nn.Module):
    def __init__(self):
        super(LightweightCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Create an instance of the lightweight CNN model
model = LightweightCNN()
print(model)

## Step 3: Train the Model

Train the lightweight CNN model using the training dataset with techniques such as learning rate scheduling and model checkpointing.

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Train the model
model.to(device)
num_epochs = 10
best_val_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')
    scheduler.step()
    
    # Save the model if validation loss has decreased
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

## Step 4: Evaluate the Model

Evaluate the performance of the trained model on the test dataset.

In [ ]:
# Load the best model
best_model = LightweightCNN()
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.to(device)

# Evaluate the model
best_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

## Step 5: Add New Features

Add your implementation for the following new features:

### Feature 1: Data Augmentation

Implement data augmentation techniques for the training data to improve model generalization.

In [ ]:
# Define data augmentation transformations for training data
# YOUR CODE HERE

### Feature 2: Learning Rate Scheduling and Training Epochs

Implement learning rate scheduling during training to adjust the learning rate based on the training progress. Find a proper number of training epochs.

In [ ]:
# Define learning rate scheduler and the number of epochs
# YOUR CODE HERE

### Feature 3: Improved Model Architecture 

Improve your design of model architecture to enhance the capability of the classification model. 

In [ ]:
# Improve the model architecture 
# YOUR CODE HERE

### Feature 4: Evaluation of A Pre-trained Model

Create a ResNet-50 model and load the ImageNet pre-trained parameters. Test the model with a 10-category test dataset Imagenette. 

In [ ]:
import os
r50_model=None # Add your implementation to create ResNet-50 with pre-trained parameters

r50_model.to(device) # Move the model to GPU is applicable
# The output size of the model is 1000 for the ImageNet dataset provides images of 1000 categories
# The indices of the 10 categories of Imagenette in the 1k ImageNet categories are
indices=[0, 217, 482, 491, 497, 566, 569, 571, 574, 701]

# Create the dataset
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
]) # A more commonly used normalization

to_download= not os.path.exists('./data/imagenette2-320')
test_dataset = datasets.Imagenette(root='./data', split='val',download=to_download,size='320px', transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the model on the test dataset
r50_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = r50_model(images)
        outputs = None # Add your implementation to get the classification score of the 10 categories from the 1000-sized outputs
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')